### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [4]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES: 
from fastbook import *                              # Getting all the Libraries. 
from fastai.callback.fp16 import *
from fastai.text.all import *                       # Getting all the Libraries.

### **GETTING THE DATA:**
- I will use **Human Numbers** dataset here. It contains the first 10000 numbers written out in English. 

In [6]:
#@ GETTING THE DATASET: 
path = untar_data(URLs.HUMAN_NUMBERS)               # Path to the Dataset. 
path.ls()                                           # Inspecting the Dataset. 

(#2) [Path('/root/.fastai/data/human_numbers/train.txt'),Path('/root/.fastai/data/human_numbers/valid.txt')]

In [7]:
#@ JOINING AND INSPECTING THE DATASET: 
lines = L()                                         # Initializing a List. 
with open(path/"train.txt") as f:                   # Opening the File. 
    lines += L(*f.readlines())                      # Reading the Lines. 
with open(path/"valid.txt") as f:                   # Opening the File. 
    lines += L(*f.readlines())                      # Reading the Lines. 
lines                                               # Inspection. 

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [9]:
#@ PREPARING THE DATASET: 
text = " . ".join([l.strip() for l in lines])       # Preparing the Dataset. 
text[:100]                                          # Inspection. 

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

**Note:**
- I will tokenize the dataset by splitting on spaces. Then I will create a list of unique tokens called vocab for **Numericalization**. Then I will convert the tokens into numbers by looking up in the index of each in the vocab. 

In [10]:
#@ TOKENIZING THE DATASET: 
tokens = text.split(" ")                            # Splitting into Tokens. 
tokens[:10]                                         # Inspecting Tokens. 

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

In [11]:
#@ GETTING UNIQUE TOKENS: 
vocab = L(*tokens).unique()                         # Getting Unique Tokens. 
vocab                                               # Inspection. 

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

In [12]:
#@ CONVERTING TOKENS INTO NUMBERS: 
word2idx = {w:i for i,w in enumerate(vocab)}       # Getting Index of Tokens. 
nums = L(word2idx[i] for i in tokens)              # Converting into Numbers. 
nums                                               # Inspection.  

(#63095) [0,1,2,1,3,1,4,1,5,1...]

### **LANGUAGE MODEL FROM SCRATCH:**
- Here I will create a list of every sequence of three words as independent variables and the next word after each sequence as the dependent variable. 

In [16]:
#@ CREATING SEQUENCE OF TOKENS: 
L((tokens[i:i+3], tokens[i+3]) for i in range(0, len(tokens)-4, 3))

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

In [17]:
#@ CREATING SEQUENCE OF TENSORS FOR NUMERICALIZED VALUES: 
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0, len(nums)-4, 3))   # Creating Sequence.
seqs                                                                           # Inspection.  

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]